In [1]:
import numpy as np
import scipy as sp
import matplotlib.pyplot as plt
%matplotlib inline

例えばこの本は，「機械学習」と「Python」の両方と関係があるが，(フラット)クラスタリングではどちらか1つにしか所属できない．  
そこで今回は，データを「トピック」と呼ばれるいくつかの小さなグループに割り当てる方法について学ぶ．  
機械学習において，このような分野を**トピックモデル(topic modeling)**と呼ぶ．

# 潜在的ディリクレ配分法(LDA, Latent Dirichlet Allocation)
skleanのlda(Linear Discriminant Analysis, 線形判別分析)とは違うので注意  
最も単純なトピックモデル．  
LDAは文章製造機のようなもので，固定のトピックをいくつか持っている．  
例えば
- 機械学習
- Python
- 料理

これらのトピックにはそれぞれ関連する単語のリストがある．  
その単語の使用されている量によってトピックが決まる．  
ここでは，この文章製造機をテキストデータの集まりから作り出し，どのようなトピックが存在するのか，そして各文書がどのトピックに割り当てられるのかを解明する．  

## トピックモデルの作成
sklearnはLDAをサポートしていないので，gensimパッケージを利用する．  
```
$ pip install gensim
```
ここでは標準的なニュースレポートのデータセット，Associated Press (AP)のデータセットを用いる．  
とりあえずデータが404になっているのでコードの実行は諦める

In [4]:
from gensim import corpora, models, similarities

In [ ]:
corpus = corpora.BleiCorpus('./data/ap/ap.bat', './data/ap/vocab.txt') # テキストデータからBleiのLDA-C形式によるコーパスを生成
model = models.ldamodel.LdaModel(corpus, num_topics=100, id2word=corpus.id2word) # コーパスからLDAモデルを取得
topics = [model[c] for c in corpus] # モデルの各単語がどのトピックに属しているかを見る. (topic index, topic_weight)が一つのドキュメントに複数
print(topics[0]) # 例: [(3, 0.023), (13, 0.116), (19, 0.075), (92, 0.107)]

割り当てられるトピックの数は少ないので，そのベクトルは疎なベクトルであると言える．

このままだと，トピックの数は10個程度になるが，モデル生成時にalphaを設定することでその閾値を変えられる．  
alphaは1より小さな正の値にするのが普通で，大きいとドキュメントごとのトピックは増え，小さいとドキュメントごとのトピックが減る．   
標準では，alpha = 1.0 / len(corpus)になっている．

In [ ]:
model = models.ldamodel.LdaModel(corpus, num_topics=100, id2word=corpus.id2word, alpha=1) # alphaを設定

トピックの正体は単語についての多項分布  
各単語のそのトピックとの関連の深さを確率で表現している．  
なので，そのトピックが何を意味しているのかは，高い確率を持つ単語のリストを提示し，人間が読み取る．  
<br>
この多項分布を元に，確率の高いものは大きく，そうでないものは小さく可視化した「ワードクラウド」を作り出すことができる．  
ワードクラウドを作るサービスには，[Wordle](http://www.wordle.net/)がある．  
WordleはWebサービスのAPIを提供しており，Pythonなどから自動でワードクラウドを作成することが可能である．

## トピック空間で類似度の比較を行う
トピックは，ワードクラウドによる要約など，それだけでも実用的である．  
<br>
トピックを応用して，トピック空間で文書の比較を行うこともできる．  
2つの文書が同じトピックについて論じられていれば，それは似ている文書であると考えることができる．  
<br>
これによって，共通する単語があまり使われていなくても，実際は同じトピックを扱っている場合にたいおうできる.  
例えば，アメリカ大統領と記述している文書とバラク・オバマと記述している文書を似ていると判断することができる．  
<br>
このように，トピックモデルはそのまま可視化やデータ把握に使えるだけではなく，他の多くの課題について，中間的な役割を担うこともできる．

前章で取り組んだ，2つの文書の比較問題をトピックモデルで解き直す．  
2つの文書の比較にトピックベクトルを用いる．  
トピック数(ここでは100）は文書に使われる単語数より少ないため，ドキュメントを次元削減していることになり，これは重要なテーマである．  

データの取得

In [13]:
import os
from sklearn.datasets import fetch_20newsgroups

if os.name == "nt":
    data_home = "D:\Programming\machinelearning\BuildingMachineLearningSystems\data" # for windows
else: 
    data_home = "/Users/komatsu/programing/machinelearning/BuildingMachineLearningSystems/data" # for Mac
    
categories = ['comp.graphics','comp.os.ms-windows.misc','comp.sys.ibm.pc.hardware','comp.sys.mac.hardware','comp.windows.x','sci.space']
data = fetch_20newsgroups(data_home=data_home, categories=categories) # 最初は数分かかる

In [ ]:
import nltk.corpus
import milk
import numpy as np
import string
from gensim import corpora, models, similarities
import sklearn.datasets
import nltk.stem
from collections import defaultdict

english_stemmer = nltk.stem.SnowballStemmer('english')
stopwords = set(nltk.corpus.stopwords.words('english'))
stopwords.update(['from:', 'subject:', 'writes:', 'writes'])


class DirectText(corpora.textcorpus.TextCorpus):

    def get_texts(self):
        return self.input

    def __len__(self):
        return len(self.input)

dataset = sklearn.datasets.load_mlcomp("20news-18828", "train", mlcomp_root='../data')
otexts = dataset.data
texts = dataset.data

texts = [t.decode('utf-8', 'ignore') for t in texts]
texts = [t.split() for t in texts]
texts = [map(lambda w: w.lower(), t) for t in texts]
texts = [filter(lambda s: not len(set("+-.?!()>@012345689") & set(s)), t) for t in texts]
texts = [filter(lambda s: (len(s) > 3) and (s not in stopwords), t) for t in texts]
texts = [map(english_stemmer.stem, t) for t in texts]
usage = defaultdict(int)

for t in texts:
    for w in set(t):
        usage[w] += 1

limit = len(texts) / 10
too_common = [w for w in usage if usage[w] > limit]
too_common = set(too_common)
texts = [filter(lambda s: s not in too_common, t) for t in texts]

corpus = DirectText(texts)
dictionary = corpus.dictionary

try:
    dictionary['computer']
except:
    pass

model = models.ldamodel.LdaModel(
    corpus, num_topics=100, id2word=dictionary.id2token)

thetas = np.zeros((len(texts), 100))

for i, c in enumerate(corpus):
    for ti, v in model[c]:
        thetas[i, ti] += v

distances = milk.unsupervised.pdist(thetas)
large = distances.max() + 1
for i in xrange(len(distances)):
    distances[i, i] = large

print(otexts[1])
print()
print()
print(otexts[distances[1].argmin()])